# The 21cm background with 21cmFAST

In this tutorial we will learn how to simulate the 21cm background with a popular semi-numerical code. We are going to: 

1) Derivation of co-eval signal cubes

2) Derivation of lightcones of the 21cm signal

3) Analysis: 21cm power spectrum

This tutorial follows part of the tutorials here: https://github.com/21cmfast/21cmFAST/tree/master/docs/tutorials
As well as descriptions here: https://21cmfast.readthedocs.io/en/latest/tutorials

### Code repository
More information on the semi-numerical code 21cmFAST can be found here:

https://github.com/21cmfast/21cmFAST

https://21cmfast.readthedocs.io

### Publications
21cmFAST was introduced here: https://arxiv.org/pdf/1003.3878.pdf

as well as its python-wrapped version: https://doi.org/10.21105/joss.02582 

In [ ]:
# some packages that we will need
#uncomment on google colab:
#!pip install -q condacolab
#import condacolab
#condacolab.install()
#!conda --version
#!conda install -c conda-forge 21cmFAST

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
# We change the default level of the logger so that
# we can see what's happening with caching.
import logging, sys, os
logger = logging.getLogger('21cmFAST')
logger.setLevel(logging.INFO)

import py21cmfast as p21c

# For plotting the cubes, we use the plotting submodule:
from py21cmfast import plotting

# For interacting with the cache
from py21cmfast import cache_tools

print(f"Using 21cmFAST version {p21c.__version__}")

In [ ]:
# Clear the cache
if not os.path.exists('_cache'):
    os.mkdir('_cache')

p21c.config['direc'] = '_cache'
cache_tools.clear_cache(direc="_cache")

## 1) Derivation of co-eval signal cubes

Co-eval cubes are boxes taken at at fixed time, or redshift. The run_coeval method performs all necessary simulation steps to derive the 21cm co-eval box. 

In [ ]:
# call run_coeval at three different redshifts
coeval8, coeval9, coeval10 = p21c.run_coeval(
    redshift = [8.0, 9.0, 10.0],
    user_params = {"HII_DIM": 100, "BOX_LEN": 100, "USE_INTERPOLATION_TABLES": True},
    cosmo_params = p21c.CosmoParams(SIGMA_8=0.8),
    astro_params = p21c.AstroParams({"HII_EFF_FACTOR":20.0}),
    random_seed=12345
)


We now have a look at the set of parameters we can access.

### Task 1: 
Explore user_params, cosmo_params, astro_params:

Create some more co-eval cubes in different settings.


In [ ]:
# you can print out the parameter set, and call help to learn more

print(coeval8.user_params)
#help(p21c.AstroParams)

Let's now have a look at the cubes!

###  Question: 
What dimensions do the co-eval cubes created have?

### Task 2:
Plot some co-eval cubes.

In [ ]:
# base example
print(coeval8.brightness_temp.shape)

fig, ax = plt.subplots(1,3, figsize=(14,4))
for i, (coeval, redshift) in enumerate(zip([coeval8, coeval9, coeval10], [8,9,10])):
    plotting.coeval_sliceplot(coeval, ax=ax[i], fig=fig);
    plt.title("z = %s"%redshift)
plt.tight_layout()

# which data boxes can exist?
print(p21c.wrapper.get_all_fieldnames(coeval8))

# choose another kind to plot
fig, ax = plt.subplots(1,3, figsize=(14,4))
for i, (coeval, redshift) in enumerate(zip([coeval8, coeval9, coeval10], [8,9,10])):
    plotting.coeval_sliceplot(coeval, kind='density', ax=ax[i], fig=fig);
    plt.title("z = %s"%redshift)
plt.tight_layout()

In [ ]:
# bonus: more functionalities are implemented, such as calculating the average, global 21cm signal
# _struct attributes contain the numpy arrays of the data cubes
print(coeval8.brightness_temp_struct.global_Tb)
print(coeval9.brightness_temp_struct.global_Tb)
print(coeval10.brightness_temp_struct.global_Tb)

### Question: 
What does run_coeval() calculate? 

Let's try to access each field individually!

In [ ]:
# default parameters
p21c.CosmoParams._defaults_

# initial density and velocity fields
initial_conditions = p21c.initial_conditions(
    user_params = {"HII_DIM": 100, "BOX_LEN": 100},
    cosmo_params = p21c.CosmoParams(SIGMA_8=0.8),
    random_seed=54321
)


print(initial_conditions.cosmo_params)
print(initial_conditions.fieldnames)

plotting.coeval_sliceplot(initial_conditions, "hires_density")

In [ ]:
# perturbed density and velocity fields (at given redshift)

perturbed_field = p21c.perturb_field(
    redshift = 8.0,
    init_boxes = initial_conditions
)

plotting.coeval_sliceplot(perturbed_field, "density")
plotting.coeval_sliceplot(perturbed_field, "velocity")


In [ ]:
# ionisation field (assuming post-heating)

ionized_field = p21c.ionize_box(
    perturbed_field = perturbed_field
)

plotting.coeval_sliceplot(ionized_field, "xH_box")

In [ ]:
# 21cm brightness offset temperature

brightness_temp = p21c.brightness_temperature(
    ionized_box=ionized_field, perturbed_field=perturbed_field
)

plotting.coeval_sliceplot(brightness_temp)

In [ ]:
# Bonus: calculate the brightness temperature without assuming post-heating
# Beware, this is much more computationally expensive

spin_temp = p21c.spin_temperature(
    perturbed_field = perturbed_field,
    zprime_step_factor=1.05,
)

plotting.coeval_sliceplot(spin_temp, "Ts_box")



ionized_box = p21c.ionize_box(
    spin_temp = spin_temp,
    zprime_step_factor=1.05,
)



brightness_temp = p21c.brightness_temperature(
    ionized_box = ionized_box,
    perturbed_field = perturbed_field,
    spin_temp = spin_temp
)

## 2) Derivation of lightcones of the 21cm signal

We now derive lightcones (where the 3rd dimension follows the time evolution of the signal) instead of co-eval cubes.

### Task: 
Feel free to play with settings here and plot different lightcones.

In [ ]:
# run_lightcone() basically interpolates co-eval cubes for you


lightcone = p21c.run_lightcone(
    redshift = 8.0,
    max_redshift = 10.0,
    user_params = {"HII_DIM":100, "BOX_LEN": 100},
    lightcone_quantities=("brightness_temp", 'density','xH_box'),
    global_quantities=("brightness_temp", 'density', 'xH_box'),
    direc='_cache'
)

#global_quantities = ('brightness_temp','Ts_box','xH_box',"dNrec_box",'z_re_box','Gamma12_box','J_21_LW_box',"density")

In [ ]:
plotting.lightcone_sliceplot(lightcone)

In [ ]:
# if you want to keep your simulated lightone
#filename = lightcone.save(direc='_cache')

In [ ]:
# you can access more information from the lightcone attribute now, for example:

# dict of global properties
print(lightcone.global_quantities)

## 3) (optional) Analysis: 21cm power spectra

### Task: 
Design a routine that derives 21cm power spectra from 'brightness_temp'

In [ ]:
# hint: https://21cmfast.readthedocs.io/en/latest/tutorials/relative_velocities.html#21-cm-power-spectra
# You can use the code snippets at the link above, or as well other code or python packages